In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Phase 3: Kou Replication & Checks\n",
    "\n",
    "This notebook calibrates Kou (2002) to NIFTY-like returns (synthetic fallback),\n",
    "then prices calls via Carr–Madan FFT, compares with Black–Scholes (λ→0), and\n",
    "plots a small smile around ATM."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np, pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "from src.data.io import load_returns_and_rf\n",
    "from src.models.kou_calibration import calibrate_mle\n",
    "from src.pricing.kou_cf import charfn_kou\n",
    "from src.pricing.fft_pricer import price_calls_fft\n",
    "\n",
    "ret, rf = load_returns_and_rf(use_db=False)\n",
    "res = calibrate_mle(ret.values, n_starts=5, seed=7)\n",
    "res.params, res.loglik, res.aic, res.bic"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "S0 = 19500.0\n",
    "T = 0.25\n",
    "r = float(rf.tail(1).values[-1]) if len(rf)>0 else 0.055\n",
    "params_q = dict(sigma=res.params['sigma'], lambda=res.params['lambda'], p=res.params['p'],\n",
    "                eta1=res.params['eta1'], eta2=res.params['eta2'], r=r, d=0.0)\n",
    "charfn = lambda u: charfn_kou(u, T, params_q)\n",
    "K, C = price_calls_fft(S0, T, r, charfn)\n",
    "plt.figure(); plt.plot(K, C); plt.title('Call prices via FFT (Kou)'); plt.xlabel('Strike'); plt.ylabel('Price'); plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {"display_name": "Python 3", "language": "python", "name": "python3"},
  "language_info": {"name": "python", "version": "3.11"}
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
